## 05 Merge Interim Data of all Players (FBREF and Transfermarkt)

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
import sys
sys.path.append("..")
from src.preprocessing.merge_player_data import run_merge_pipeline

merge_result: tuple = run_merge_pipeline("Valencia CF", ["2223", "2324", "2425"])
df_all_players = merge_result[0] if isinstance(merge_result, tuple) else merge_result

print(f"Merge completed successfully!")
print(f"Total merged rows: {df_all_players.shape[0]}")
print(f"Total columns: {df_all_players.shape[1]}")

In [ ]:
total_with_market_value = df_all_players['MarketValueEuro'].notna().sum()
print(f"Rows with market value data: {total_with_market_value}/{len(df_all_players)} ({total_with_market_value/len(df_all_players)*100:.1f}%)")

In [ ]:
print("\nSample merged data:")
sample_cols = ['Player', 'Season', 'MarketValueEuro', 'Age_x', 'Position']
available_cols = [col for col in sample_cols if col in df_all_players.columns]
display(df_all_players[available_cols].head(20))

In [ ]:
# Group by player and sort by season to see all seasons for each player with missing market values
players_with_missing_market_values = df_all_players[df_all_players['MarketValueEuro'].isna()].sort_values(['Player', 'Season'])
display(players_with_missing_market_values[['Player', 'Season', 'Age', 'Position']].head(40))

In [6]:
# Remove system/total rows
df_clean = df_all_players[~df_all_players['Player'].str.contains('Total|Squad|Opponent', case=False)]

# Only players with market values
df_with_market_values = df_all_players[df_all_players['MarketValueEuro'].notna()]

In [ ]:
df_with_market_values

NOTE: Current issues: 
- Nation and Nationality contain different values
- Pos and Position are not the same and we have in Pos sometimes multiple values 
    * By keeping only last 3 characters in 'Pos' the data is cleaner and compact
- We have new players with only one season or missing values 
    * Let's skip them then, it's best to look at players with 2-3+ seasons

In [ ]:
#### Let's use this column instead of 'Pos' since it's cleaner
df_with_market_values['Position'].value_counts()

In [ ]:
df_with_market_values['Nation'].value_counts().head(3)

In [ ]:
df_with_market_values['Nation'] = df_with_market_values['Nation'].str[-3:]

In [ ]:
print(df_with_market_values.shape)
df_with_market_values.head(10)

In [ ]:
df_with_market_values['Nation'].value_counts().head(3)

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

df_with_market_values['Nation'] = le.fit_transform(df_with_market_values['Nation'])
df_with_market_values['Position'] = le.fit_transform(df_with_market_values['Position'])

In [ ]:
correlation_with_market_value = df_with_market_values.corrwith(df_with_market_values['MarketValueEuro'])

print(correlation_with_market_value.sort_values(ascending=False).head(11))

### 🔟 Top 10 Features Most Correlated with Market Value

- **Starts (0.718)** — Indicates how often a player is trusted to start matches; more starts = more value.
- **90s (0.693)** — Reflects total playing time in 90-minute equivalents; regular play boosts visibility and importance.
- **Min (0.693)** — Total minutes played; more minutes typically means more experience and trust from coaches.
- **Cmp.3 (0.645)** — Completed passes in the attacking third; shows involvement in crucial offensive phases.
- **PrgDist (0.642)** — Total progressive carrying distance; highlights ability to move the ball forward effectively.
- **Att.3 (0.638)** — Passes attempted in the attacking third; shows a player’s role in build-up and attacking moves.
- **Def_Pen (0.621)** — Defensive actions inside the penalty area; key for evaluating defenders’ last-line contributions.
- **MP (0.610)** — Matches played; simple yet powerful indicator of consistency and fitness.
- **TotDist (0.592)** — Total distance covered while carrying the ball; proxies physical activity and involvement.
- **Carries (0.568)** — Number of ball carries; emphasizes players who actively retain and progress possession.


- Nation and Position don't have an impact on Market Value

### Interrogations:
- When scraping data from other teams to build a regression model to predict a player's next year's market value, I believe we should consider the ranking of the team. 
- A player for Valencia might not be value the same as a Real Madrid player even with the same performance.
- That's because Real Madrid plays bigger competitions and is more likely to win titles which boosts the player's exposure and therefore the market price.
- number of competitions played as well

## Other spanish teams to scrape for ML:

- https://fbref.com/en/squads/53a2f082/2024-2025/Real-Madrid-Stats
- https://fbref.com/en/squads/206d90db/Barcelona-Stats
- https://fbref.com/en/squads/ad2be733/Sevilla-Stats
- https://fbref.com/en/squads/db3b9613/Atletico-Madrid-Stats
- https://fbref.com/en/squads/2b390eca/Athletic-Club-Stats
- https://fbref.com/en/squads/2a8183b3/Villarreal-Stats
- https://fbref.com/en/squads/e31d1cd9/Real-Sociedad-Stats
- https://fbref.com/en/squads/fc536746/Real-Betis-Stats

---

# Merge all player data

In [1]:
import sys
import os
from pathlib import Path
sys.path.append("..")
from src.preprocessing.merge_player_data import run_merge_pipeline

teams_list: list[str] = [
    "Real Madrid CF",
    "FC Barcelona", 
    "Atlético Madrid",
    "Sevilla FC", 
    "Athletic Club",
    "Villarreal CF",
    "Real Sociedad",
    "Real Betis",
    "Valencia CF",
]

seasons_list: list[str] = ["2223", "2324", "2425"]

for team_name in teams_list:
    print(f"Processing {team_name}...")
    
    # Create directory structure
    team_directory: Path = Path(f"../data/interim/{team_name}/merged")
    team_directory.mkdir(parents=True, exist_ok=True)
    
    # Run merge pipeline for current team
    merge_result: tuple = run_merge_pipeline(team_name, seasons_list)
    df_all_players = merge_result[0] if isinstance(merge_result, tuple) else merge_result
    
    print(f"Merge completed for {team_name}!")
    print(f"Total merged rows: {df_all_players.shape[0]}")
    print(f"Total columns: {df_all_players.shape[1]}")
    
    # Remove system/total rows
    player_column_filled = df_all_players['Player'].fillna('Unknown')
    df_clean = df_all_players[~player_column_filled.str.contains('Total|Squad|Opponent', case=False)]

    # Only players with market values
    df_clean = df_clean[df_clean['MarketValueEuro'].notna()]
    
    # Clean nation column
    df_clean.loc[:, 'Nation'] = df_clean['Nation'].str.extract(r'([A-Z]{3})$').iloc[:, 0]

    # Save processed data
    clean_filename: str = f"{team_name.replace(' ', '_').lower()}_clean.csv"
    
    df_clean.to_csv(team_directory / clean_filename, index=False)
    
    print(f"Saved {clean_filename} to {team_directory}")
    print("-" * 50)

print("All teams processed successfully!")

/Users/markuskuehnle/Documents/projects/talent-value-forecast/.venv/lib/python3.10/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


Processing Real Madrid CF...
Current working directory: /Users/markuskuehnle/Documents/projects/talent-value-forecast/notebooks
Found project root: /Users/markuskuehnle/Documents/projects/talent-value-forecast
Trying possible paths: ['/Users/markuskuehnle/Documents/projects/talent-value-forecast/data', '../data', '../../data', '/Users/markuskuehnle/Documents/projects/talent-value-forecast/data', '/Users/markuskuehnle/Documents/projects/data', 'data']
Checking path: /Users/markuskuehnle/Documents/projects/talent-value-forecast/data - exists: True
Found data directory: /Users/markuskuehnle/Documents/projects/talent-value-forecast/data
Looking for FBref directory: /Users/markuskuehnle/Documents/projects/talent-value-forecast/data/interim/Real Madrid CF/fbref
Looking for Transfermarkt file: /Users/markuskuehnle/Documents/projects/talent-value-forecast/data/interim/Real Madrid CF/transfermarkt/real_madrid_cf_2020_2024.csv
Attempting fuzzy matching for 41 unmatched players...
Fuzzy match: 'v

---

# Concatenate all cleaned team datasets

In [9]:
# Concatenate all cleaned team datasets
import pandas as pd
from pathlib import Path

# Find all clean CSV files in the merged directories
merged_data_path: Path = Path("../data/interim")
clean_files_pattern: str = "*/merged/*_clean.csv"

all_clean_files: list[Path] = list(merged_data_path.glob(clean_files_pattern))

print(f"Found {len(all_clean_files)} clean CSV files:")
for file_path in all_clean_files:
    print(f"  - {file_path}")

# Load and concatenate all datasets
team_dataframes: list[pd.DataFrame] = []
for file_path in all_clean_files:
    team_df: pd.DataFrame = pd.read_csv(file_path)
    team_name: str = file_path.parent.name  # Extract team name from directory
    team_dataframes.append(team_df)

# Concatenate all dataframes
combined_dataset: pd.DataFrame = pd.concat(team_dataframes, ignore_index=True)

print(f"\nCombined dataset shape: {combined_dataset.shape}")
print(f"Seasons included: {sorted(combined_dataset['Season'].unique())}")

# Save combined dataset
output_path: Path = Path("../data/interim/datasets_combined")
output_path.mkdir(parents=True, exist_ok=True)
combined_filename: str = "all_teams_combined.csv"
combined_dataset: pd.DataFrame = combined_dataset.drop(columns=['Nation'])
combined_dataset.to_csv(output_path / combined_filename, index=False)

print(f"\nCombined dataset saved to: {output_path / combined_filename}")

Found 9 clean CSV files:
  - ../data/interim/Atlético Madrid/merged/atlético_madrid_clean.csv
  - ../data/interim/Valencia CF/merged/valencia_cf_clean.csv
  - ../data/interim/Sevilla FC/merged/sevilla_fc_clean.csv
  - ../data/interim/Villarreal CF/merged/villarreal_cf_clean.csv
  - ../data/interim/Real Madrid CF/merged/real_madrid_cf_clean.csv
  - ../data/interim/Real Betis/merged/real_betis_clean.csv
  - ../data/interim/FC Barcelona/merged/fc_barcelona_clean.csv
  - ../data/interim/Athletic Club/merged/athletic_club_clean.csv
  - ../data/interim/Real Sociedad/merged/real_sociedad_clean.csv

Combined dataset shape: (624, 122)
Seasons included: [np.int64(2223), np.int64(2324), np.int64(2425)]

Combined dataset saved to: ../data/interim/datasets_combined/all_teams_combined.csv


In [10]:
combined_dataset.head(30)

,Player,Season,Nationality,Pos,Position,Age,MP,Starts,Min,90s,...,Succ%,Tkld,Tkld%,Carries,CPA,Mis,Dis,Rec,MarketValueEuro,Current club
0,Abdellah Raihani,2324,Morocco,FW,Centre-Forward,19,1,0,10.0,0.1,...,NaN,0.0,NaN,1.0,0.0,1.0,0.0,1.0,300000.0,Atlético Madrid
1,Adrian Niño,2324,Spain,FW,Centre-Forward,19,0,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,500000.0,Atlético Madrid
2,Aitor Gismera,2324,Spain,MF,Central Midfield,19,0,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,250000.0,Atlético Madrid
3,Antoine Griezmann,2324,France,"FW,MF",Centre-Forward,32,33,31,2644.0,29.4,...,57.9,15.0,39.5,839.0,7.0,40.0,26.0,1194.0,25000000.0,Atlético Madrid
4,Antonio Gomis,2324,Spain,GK,Goalkeeper,20,0,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,150000.0,Atlético Madrid
5,Arthur Vermeeren,2324,Belgium,MF,Defensive Midfield,18,5,2,162.0,1.8,...,0.0,1.0,100.0,68.0,0.0,6.0,2.0,75.0,25000000.0,Atlético Madrid
6,Axel Witsel,2324,Belgium,DF,Centre-Back,34,35,30,2783.0,30.9,...,62.5,3.0,37.5,1364.0,0.0,14.0,6.0,1467.0,3500000.0,Atlético Madrid
7,César Azpilicueta,2324,Spain,DF,Centre-Back,33,25,14,1423.0,15.8,...,16.7,3.0,50.0,624.0,3.0,10.0,6.0,752.0,2000000.0,Atlético Madrid
8,Gabriel Paulista,2324,Brazil,DF,Centre-Back,32,5,5,405.0,4.5,...,NaN,0.0,NaN,187.0,0.0,0.0,1.0,212.0,2000000.0,Atlético Madrid
9,Horaţiu Moldovan,2324,Romania,GK,Goalkeeper,25,0,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2500000.0,Atlético Madrid


In [ ]:
- NOTE: 